In [ ]:
#!pip install -q pyyaml h5py
#!pip install imageai
#!pip install silence-tensorflow

In [17]:
import datetime as dt
t_start_abs = dt.datetime.now()

In [18]:
import os
import gzip
import zipfile
import imageio #convert to image
import numba
import zipfile

# import silence_tensorflow.auto
import nibabel as nib
from scipy import ndimage
import scipy.special as sc
# import numba_special

import numpy as np
# import numba_scipy
from numba import jit
from numba import njit
import threading as tr

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from imageai.Classification.Custom import CustomImageClassification
### numba warrnings ###########
from numba import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

# tf.logging.set_verbosity(tf.logging.ERROR)
# print(tf.version.VERSION)

In [19]:
PROJECT_DIR = os.getcwd()
# PROJECT_DIR

In [20]:
t_start = dt.datetime.now()

def get_model_RN50(image):

    execution_path = os.getcwd()
#     print(execution_path)
    path_to_model = '/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/model/model_ex-083_acc-0.951409.h5'
    path_to_classes = '/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/json/model_class.json'
#     print(path_to_model)
#     print(path_to_classes)

    prediction = CustomImageClassification();
    prediction.setModelTypeAsResNet50();
    prediction.setModelPath(os.path.join(execution_path, path_to_model));
    prediction.setJsonPath(os.path.join(execution_path, path_to_classes)); #"model_class.json"
    prediction.loadModel(num_objects = 2);
    predictions, probabilities = prediction.classifyImage(image, result_count=5)

    
    return predictions, probabilities

t_finish = dt.datetime.now() - t_start
t_finish.seconds

0

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
t_start = dt.datetime.now()

### создаем энкодер для файлов gzip
def encoder_gzip2nii_(path, output_path = None):
    global PROJECT_DIR
    os.chdir(path)
    for item in os.listdir(path):
        print(item)
        if (item.find('.')) > 0:
            # print(str(item.find('.')))
            continue
        else:
            if item.endswith(".gzip"):
                print('Dot not find, GZIP find')
                input = gzip.GzipFile(item)
                # print(item)
                data = input.read()
                input.close()
            elif item.endswith(".zip"):
                print('Dot not find, ZIP find')
                input = zipfile.ZipFile.extract(item)
                # print(item)
                data = input.read()
                input.close()
            else:
                continue
            if output_path == None:
                execution_path = os.getcwd()
                output_path = execution_path + '/MRI'
#                 print(f"Output path: {output_path}")
                file_output = output_path + item + ".nii"
                output = open(file_output, 'wb')
                output.write(data)
                output.close()
            else:
                file_output = output_path + item + ".nii"
                output = open(file_output, 'wb')
                output.write(data)
                output.close()
#             print(file_output)
            if image_folder == None:
                image_folder = PROJECT_DIR + "nii_to_jpg/" + item + "/"
                image = nii_to_image_(output_path, image_folder = image_folder);
            os.remove(item)
            
            return image
# dir_name_gzip = '/home/sergio/JN/NN/COVID_detection_picture/GZIP/'
# encoder_gzip2nii_(dir_name_gzip, output_path = None)
# encoder_gzip2nii_(dir_name_healthy, output_path = None)

t_finish = dt.datetime.now() - t_start
t_finish.seconds

0

In [23]:
t_start = dt.datetime.now()

def nii_to_image_(file_folder, image_folder = None):
    global PROJECT_DIR
    file_folder = file_folder.rstrip(file_folder.split("/")[-1])
    # print(f"File NII folder: {file_folder}")
    
    filenames = os.listdir(file_folder) #read nii folder
    # print(f"File (-'s) NII in folder: {filenames}")
    
    for f in filenames:
        if f.endswith(".nii"):
            #Start reading nii files
            img_path = os.path.join(file_folder, f)
            img = nib.load(img_path); #read nii
            img_fdata = img.get_fdata();
            fname = f.replace('.nii', '') # Remove the nickname of nii
            # img_f_path = os.path.join(image_folder, fname) #Create a folder corresponding to the image of nii
            if image_folder == None:
                image_folder = PROJECT_DIR + "/nii_to_jpg/" + f + "/"
            if not os.path.exists(image_folder):
                os.mkdir(image_folder) #New folder
    
            #  to image
            (x,y,z) = img.shape
            for i in range(z): #z is the sequence of images
                slider = img_fdata[:, :, i] #Select which direction the slice can be
                imageio.imwrite(os.path.join(image_folder, fname + "_slice_" + str(i) + '.jpg'), slider); # save image
#             os.remove(os.path.join(image_folder, f))
#             print(f"This is ImageFolder with PNG: {image_folder}")
            return image_folder

# dir_name_healthy = "/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/dataset/healthy"
# dir_name_sick = "/content/drive/MyDrive/JN/NN/COVID_detection_picture/Keras/dataset/sick"
    # filepath = "/content/drive/MyDrive/JN/NN/COVID_detection_picture/dataset/test_gunzip/" #7e55febce08fc5f0e53f6a6e6746e7a0.nii" #nii folder
    # image_folder = "/content/drive/MyDrive/JN/NN/COVID_detection_picture/dataset/test_gunzip/jpg/"
# nii_to_image_(dir_name_healthy);
t_start = dt.datetime.now()
t_finish = dt.datetime.now() - t_start
t_finish.seconds

0

In [24]:
dir_path = '/content/drive/MyDrive/JN/NN/COVID_detection_picture/dataset/val/sick/'
# lst_images = ['sick_7c983eabf45915f3e7e9cae2aa9a5569.nii',
#               'sick_7ff18f5d3de11b9ae7a9e5d651313fbd.nii',
#               'healthy_2dfd2d7087aaee9777bf6ea42de958dc.nii',
#               'healthy_0a38b4c71a46ce68c8b48c95b93619b2.nii'
#              ]

In [25]:
# os.path.join(PROJECT_DIR, lst_images[0])

In [26]:
# os.path.join(dir_path + lst_images[0])
# (os.path.join('sick_7c983eabf45915f3e7e9cae2aa9a5569')).endswith("")

In [27]:
t_start = dt.datetime.now()
# class_names = ["normal", "abnormal"]
def main(path):
    global PROJECT_DIR
    execution_path = os.getcwd()
    if path.endswith(".nii"):
        image_folder = nii_to_image_(path, image_folder = None);
    elif path.endswith(".gzip") or path.endswith(".gzip"):
        image_folder = encoder_gzip2nii_(path, output_path = None);
    elif path.endswith(".jpg") or path.endswith(".png"):
        image_folder = path
    elif path.endswith(""):
        try:
            encoder_gzip2nii_(path, output_path = None);
            image_folder = encoder_gzip2nii_(path, output_path = None);
        except:
            return 'File have dont readable format. Readable: JPG, PNG, ZIP, GZIP'
#     print(f"------------------------{image_folder}----------------")
    summary_accuracy_class_1 = np.empty(0)
    summary_accuracy_class_2 = np.empty(0)
    classes = []
    result = dict()
    for item in os.listdir(image_folder):
        print(f"-------Image PNG:{item}------------")
        predictions, probabilities = get_model_RN50(os.path.join(path, item))
        for eachPrediction, eachProbability in zip(predictions, probabilities):
            print(f'Class: {eachPrediction} | Rate: {eachProbability}')
            if eachPrediction not in classes:
                classes.append(eachPrediction)
            if eachPrediction == classes[0]:
                summary_accuracy_class_1 = np.append(summary_accuracy_class_1, float(eachProbability))
            elif eachPrediction == classes[1]:
                summary_accuracy_class_2 = np.append(summary_accuracy_class_2, float(eachProbability))
    A = np.mean(summary_accuracy_class_1)
    result[classes[0]] = str(A / 100)
    B = np.mean(summary_accuracy_class_2)
    result[classes[1]] = str(B / 100)
    return result

# print(os.path.join(dir_path + lst_images[0]))
# result = main(os.path.join(dir_path + lst_images[0]))
# result = main(os.path.join(dir_path + "sick_7ff18f5d3de11b9ae7a9e5d651313fbd.nii"));
result = main(os.path.join(dir_path));
print(result)

t_finish = dt.datetime.now() - t_start
t_finish.seconds

1444ff0fc8287291504b3e44a8e571f1_slice_35.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_22.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_28.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_4.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_18.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_7.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_32.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_12.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_3.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_27.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_1.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_13.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_9.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_37.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_15.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_26.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_44.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_29.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_36.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_10.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_31.jpg
1444ff0fc8287291504b3e44a8e571f1_slice_

295

In [28]:
result

{'healthy': '0.0', 'sick': '1.0'}

In [29]:
t_finish = dt.datetime.now() - t_start_abs
t_finish.seconds

295

In [30]:
# # test_image = path + lst_images[0]
# # prediction = loaded_model.predict(np.expand_dims(test_image, axis=0))[0]
# # scores = [1 - prediction[0], prediction[0]]
# for item in lst_images:
#     label_image = item.split("_")[0]
#     print(f"Real label: {label_image}")
#     img = nib.load(os.path.join(dir_path, item)) #read nii
#     image = img.get_fdata()
# #     imageio.imwrite()

#     plt.imshow(np.squeeze(image[:, :, 2]), cmap="gray")
#     plt.show()

In [31]:
# t_finish = dt.datetime.now() - t_start
# t_finish.seconds

In [32]:
#!ipynb-py-convert RN50_detect.ipynb RN50_detect.py

/bin/bash: ipynb-py-convert: command not found
